In [7]:
# Slightly outdated DB with ~500k games, we will use another one

# import csv, sqlite3
#con = sqlite3.connect('database.db') 
#cur = con.cursor()
#cur.execute("DROP TABLE IF EXISTS games")
#cur.execute("CREATE TABLE games (id, slug, name, metacritic, released, tba,	updated, website, rating, rating_top, playtime, achievements_count, ratings_count, suggestions_count, game_series_count, reviews_count, platforms, developers,	genres,	publishers,	esrb_rating,	added_status_yet,	added_status_owned,	added_status_beaten,	added_status_toplay, \
#added_status_dropped, added_status_playing)")

#import pandas as pd
# load the data into a Pandas DataFrame
#games_dataframe = pd.read_csv('game_info.csv')
# write the data to a sqlite table
#games_dataframe.to_sql('games_dataframe', con, if_exists='append', index = False)
#games_dataframe.head()

In [8]:
# Current database with ~100k games
import sqlite3, json
import pandas as pd
from pathlib import Path

######## Create Games database ########
p = Path('games.json')

with p.open('r', encoding='utf-8') as f:
    data = json.loads(f.read())

df = pd.json_normalize(data['data']['games']) 

# Convert lists of int in dataframe to strings so that they can be stored in the SQLite database (lists are unsupported)
# Therefore need to parse the variables that use this function when querying to reassemble the list
def convertType(var):
    df[var] = df[var].apply(str)
    df[var] = df[var].apply(''.join)

convertType('developers')
convertType('genres')
convertType('publishers')
convertType('alternates')
convertType('uids')
convertType('hashes')

# Update database
con = sqlite3.connect('database.db') 
cur = con.cursor()
#cur.execute("DROP TABLE IF EXISTS games")
#cur.execute("CREATE TABLE games (id, game_title, release_date, platform, region_id,	country_id, overview, youtube, players, coop, rating, developers, genres, publishers, alternates, uids, hashes)")

df.to_sql('games', con, if_exists='replace', index = False)
df.head()


######## Create users database ########
con = sqlite3.connect('database.db') 
cur = con.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS users (user_id, email, password, join_date)")



In [48]:
# Examples of games database query

# Example 1
# Retrieves all attributes for games with title equal to 'Fallout 3'
# Note that this produces 3 results since each different platform has an entry
print('----------EXAMPLE 1----------\n')
test_query = '''
            SELECT * FROM games
            WHERE game_title = 'Fallout 3'
            '''
cur.execute(test_query)
all_rows = cur.fetchall()
for row in all_rows:
    print(row, '\n')
    
#Example 2
# Retrieves title, and release date for all games starting with Halo
print('----------EXAMPLE 2----------\n')
test_query = '''
            SELECT DISTINCT game_title, release_date FROM games
            WHERE game_title LIKE 'Halo%'
            ORDER BY game_title
            '''
cur.execute(test_query)
all_rows = cur.fetchall()
for row in all_rows:
    print(row, '\n')

# We can also select the distinct game title for a specific game to avoid returning multiple listings because 
# the game was released on multiple platforms:
print('----------EXAMPLE 3----------\n')
test_query = '''
            SELECT DISTINCT game_title FROM games
            WHERE game_title = 'Halo 2'
            '''
cur.execute(test_query)
all_rows = cur.fetchall()
for row in all_rows:
    print(row, '\n')
# However, we still need to 

----------EXAMPLE 1----------

(86, 'Fallout 3', '2008-10-28', 1, 0, 0, 'Fallout 3 takes place in the year 2277, 36 years after the setting of Fallout 2 and 200 years after the nuclear apocalypse that devastated the game\'s world in a future where international conflicts culminated in a Sino-American war in the second half of the 21st century. The game places the player in the role of an inhabitant of Vault 101, a survival shelter the size of a village, designed to protect a small number of humans from the nuclear fallout. When the player character\'s father disappears under mysterious circumstances, the player is forced to escape from the Vault and journey into the ruins of Washington D.C. to track him down. Along the way the player is assisted by a number of human survivors and must battle a myriad of enemies that inhabit the area now known as the "Capital Wasteland".', 'iYZpR51XgW0', 1.0, 'No', 'M - Mature 17+', '[1009]', '[1, 2, 4, 8]', '[34]', 'None', 'None', 'None') 

(6600, 'Fal